In [1]:
import numpy as np
import matplotlib.pyplot as plt
import requests
import tensorflow as tf

from src.plotter.Plotter import Plotter
from scipy.optimize import curve_fit
from src.maths.Error import r_squared

In [2]:
M = 200.59e-3
R = 8.31
N_A = 6e23

In [3]:
data = requests.get('http://77.246.98.155/database/phase_diagram/?format=json').json()
temperature = np.array([dat['temperature'] for dat in data if dat['temperature']])
pressure = np.array([dat['pressure'] for dat in data if dat['temperature']])
density = np.array([dat['density'] for dat in data if dat['temperature']])

linearized_temperature = (temperature - min(temperature)) / (max(temperature) - min(temperature))
linearized_pressure = (pressure - min(pressure)) / (max(pressure) - min(pressure))
linearized_density = (density - min(density)) / (max(density) - min(density))

In [4]:
inputs = tf.keras.layers.Input(shape=(2,))
x = tf.keras.layers.Dense(10, activation='tanh')(inputs)
x = tf.keras.layers.Dense(10, activation='tanh')(x)
outputs = tf.keras.layers.Dense(1, activation='linear')(x)
model = tf.keras.models.Model(inputs=inputs, outputs=outputs)
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 2)]               0         
                                                                 
 dense (Dense)               (None, 10)                30        
                                                                 
 dense_1 (Dense)             (None, 10)                110       
                                                                 
 dense_2 (Dense)             (None, 1)                 11        
                                                                 
Total params: 151 (604.00 Byte)
Trainable params: 151 (604.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [11]:
mask = np.arange(len(linearized_temperature))
np.random.shuffle(mask)

x_train = np.array([linearized_temperature[mask], linearized_pressure[mask]]).T
y_train = np.array([linearized_density[mask]]).T
model.fit(x_train, y_train, epochs=300, validation_split=0.2)

Epoch 1/300
21/21 [==============================] - 0s 10ms/step - loss: 5.5189e-05 - accuracy: 0.0030 - val_loss: 8.6790e-05 - val_accuracy: 0.0000e+00
Epoch 2/300
21/21 [==============================] - 0s 4ms/step - loss: 5.6359e-05 - accuracy: 0.0030 - val_loss: 7.8189e-05 - val_accuracy: 0.0000e+00
Epoch 3/300
21/21 [==============================] - 0s 4ms/step - loss: 5.4618e-05 - accuracy: 0.0030 - val_loss: 8.0495e-05 - val_accuracy: 0.0000e+00
Epoch 4/300
21/21 [==============================] - 0s 4ms/step - loss: 5.1487e-05 - accuracy: 0.0030 - val_loss: 8.9385e-05 - val_accuracy: 0.0000e+00
Epoch 5/300
21/21 [==============================] - 0s 5ms/step - loss: 5.1760e-05 - accuracy: 0.0030 - val_loss: 7.9850e-05 - val_accuracy: 0.0000e+00
Epoch 6/300
21/21 [==============================] - 0s 4ms/step - loss: 5.2514e-05 - accuracy: 0.0030 - val_loss: 8.1529e-05 - val_accuracy: 0.0000e+00
Epoch 7/300
21/21 [==============================] - 0s 4ms/step - loss: 5.2545e-

In [12]:
rang = 300

temperatures, pressures = np.meshgrid(np.linspace(min(linearized_temperature), max(linearized_temperature), rang), np.linspace(min(linearized_pressure), max(linearized_pressure), rang))
temperatures = temperatures.flatten()
pressures = pressures.flatten()

x_test = np.array([temperatures, pressures]).T
y_test = model.predict(x_test)

temperatures = temperatures * (max(temperature) - min(temperature)) + min(temperature)
pressures = pressures * (max(pressure) - min(pressure)) + min(pressure)
y_test = y_test.T[0] * (max(density) - min(density)) + min(density)

2813/2813 [==============================] - 4s 1ms/step


In [15]:
%matplotlib qt
plotter = Plotter(dimension='3d')
reshaped_temperatures = np.reshape(temperatures, (rang, rang))
reshaped_pressures = np.reshape(pressures, (rang, rang))
reshaped_y_test = np.reshape(y_test, (rang, rang))
plotter.scatter(temperature, pressure, density, color='r')
plotter.plot_wireframe(reshaped_temperatures, reshaped_pressures, reshaped_y_test, color='b', alpha=0.5)
plotter.xlabel(r'$(T)$ $(K)$')
plotter.ylabel(r'$(p)$ $(Pa)$')
plotter.zlabel(r'$(\rho)$ $(kg/m^3)$')
plotter.legend(['Experimental data', 'Network data'])
# plotter.save('Network_experimental.jpg')
plotter.show()